In [1]:
cd ..

/mnt/disk2/samarth/qdpy-numpyro


In [2]:
cd qdpy_jax

/mnt/disk2/samarth/qdpy-numpyro/qdpy_jax


In [29]:
run jax_rotation_kernels.py

Omega()
JIT version is faster by 66.39498525437742
minus1pow()
JIT version is faster by:  53.871654795657285
minus1pow_vec()
JIT version is faster by:  30.94650590995218
gamma()
JIT version is faster by:  2.593262137139752
compute_Tsr()
JIT version is faster by:  115.90014538666571
Time taken per iteration (jax jitted) compute_Tsr = 5.313e-02 seconds
get_Cvec()
JIT version is faster by:  237.84513498945518
Time taken per iteration (jax jitted) get_Cvec = 4.556e-02 seconds


In [30]:
run rotation_kernels.py

[compute_Tsr] Time taken per iteration in seconds (no jax): , 4.143e-02 seconds
[get_Cvec] Time taken per iteration in seconds (no jax): , 4.921e-01 seconds


In [26]:
 5.525e-03/4.952e-04

11.157108239095315

In [1]:
pwd

'/mnt/disk2/samarth/qdpy-numpyro/notebooks'

In [2]:
cd ..

/mnt/disk2/samarth/qdpy-numpyro


In [4]:
run qdpy_jax/one_walker_one_iteration.py

-----------
SUBMAT_DICT(startx=array([[   0,    0,    0,    0,    0],
       [ 391,  391,  391,  391,  391],
       [ 786,  786,  786,  786,  786],
       [1173, 1173, 1173, 1173, 1173],
       [1572, 1572, 1572, 1572, 1572]], dtype=int32), starty=array([[   0,  391,  786, 1173, 1572],
       [   0,  391,  786, 1173, 1572],
       [   0,  391,  786, 1173, 1572],
       [   0,  391,  786, 1173, 1572],
       [   0,  391,  786, 1173, 1572]], dtype=int32), endx=array([[ 391,  391,  391,  391,  391],
       [ 786,  786,  786,  786,  786],
       [1173, 1173, 1173, 1173, 1173],
       [1572, 1572, 1572, 1572, 1572],
       [1955, 1955, 1955, 1955, 1955]], dtype=int32), endy=array([[ 391,  786, 1173, 1572, 1955],
       [ 391,  786, 1173, 1572, 1955],
       [ 391,  786, 1173, 1572, 1955],
       [ 391,  786, 1173, 1572, 1955],
       [ 391,  786, 1173, 1572, 1955]], dtype=int32))
CENMULT_AND_NBS(nl_nbs=array([[  0, 195],
       [  0, 197],
       [  0, 193],
       [  0, 199],
       [  0, 

In [2]:
import jax
def test_laxwhile(a,b):
    quantity = 0.
    i = a + b
    j = 2 * i + 1
    k = 2 * j + 1

    condfun_1 = lambda inp: inp[1] > 0. # i > 0.
    condfun_2 = lambda inp: inp[2] > 0. # j > 0.
    condfun_3 = lambda inp: inp[3] > 0. # k > 0.

    def bodyfun_1(inp):
        quantity, i, j, k = inp
        j = 2 * i + 1
        def bodyfun_2(inp):
            quantity, i, j, k = inp
            k = 2 * j + 1
            def bodyfun_3(inp):
                quantity, i, j, k = inp
                #quantity += i + j + k  # This works!
                quantity += i + k + j   # This works!
                k -= 1.
                return (quantity, i, j, k)
            result = jax.lax.while_loop(condfun_3, bodyfun_3, (quantity,i,j,k))
            quantity = result[0]
            j -= 1.
            return (quantity, i, j, k)
        result = jax.lax.while_loop(condfun_2, bodyfun_2, (quantity,i,j,k))
        quantity = result[0]
        i -= 1.
        return (quantity, i, j, k)

    result = jax.lax.while_loop(condfun_1, bodyfun_1, (quantity,i,j,k))
    return result[0]

print(test_laxwhile(3.,2.))

6060.0


In [3]:
from jax.lax import fori_loop as foril

In [24]:
k = 0

def func2(i, k):
    return k + 0.000077

def func1(i, k):
    return foril(0, 10, func2, k) + 0.33


k = foril(0, 10, func1, k)
print(k)

3.3077009


In [49]:
@jax.jit
def func3(i, arrj):
    return np.array(jax.ops.index_update(arrj,
                               jax.ops.index[0, i], i))
    
import numpy as np
import jax.numpy as jnp
a = np.random.rand(10, 10)
aj = jnp.asarray(a)
b = np.random.rand(10, 10)
bj = jnp.asarray(b)

aj = foril(0, 10, 
           lambda i, aj: \
           foril(0, 10, 
                 lambda ii, aj:
                     jax.ops.index_update(aj,
                                jax.ops.index[ii, i], i+ii), aj), aj)

sumj = np.zeros_like(b)
sumi = np.zeros_like(b)
for i in range(10):
    for ii in range(10):
        sumj[i, ii] = b[:i, ii].sum()
        sumi[i, ii] = b[i, :ii].sum()

sumi = jnp.asarray(sumi)
        
aj = foril(0, 10, 
           lambda i, aj: \
           foril(0, 10, 
                 lambda ii, aj:
                     jax.ops.index_update(aj,
                                jax.ops.index[ii, i], sumi[i, ii]), aj), aj)

In [45]:
aj

DeviceArray([[0.29211652, 0.40524694, 0.41426554, 0.13652478, 0.51138437,
              0.51138437, 0.51138437, 0.51138437, 0.51138437, 0.51138437],
             [0.208101  , 0.7591928 , 0.4839492 , 0.39776158, 0.9148982 ,
              0.9148982 , 0.9148982 , 0.9148982 , 0.9148982 , 0.9148982 ],
             [0.66985756, 0.31976643, 0.3175408 , 0.18683541, 0.6922642 ,
              0.6922642 , 0.6922642 , 0.6922642 , 0.6922642 , 0.6922642 ],
             [0.20286793, 0.49299115, 0.252254  , 0.90681934, 0.72484434,
              0.72484434, 0.72484434, 0.72484434, 0.72484434, 0.72484434],
             [0.05334908, 0.81094474, 0.10209108, 0.22457187, 0.88225615,
              0.88225615, 0.88225615, 0.88225615, 0.88225615, 0.88225615],
             [0.05334908, 0.81094474, 0.10209108, 0.22457187, 0.88225615,
              0.88225615, 0.88225615, 0.88225615, 0.88225615, 0.88225615],
             [0.05334908, 0.81094474, 0.10209108, 0.22457187, 0.88225615,
              0.88225615, 0.8822